<a href="https://colab.research.google.com/github/MatheusHenriqueDaSilva/data_science/blob/main/exerc_arvore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split


In [54]:
# Data understanding
DATA = pd.read_csv('/content/classificacao_Q2.csv')
DATA.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,target
0,-1.39917,0.96108,-0.36145,0.26092,2.78953,-1.55046,-0.79886,0.24304,1.51733,-0.03357,-1.51127,-0.97444,-0.01934,0
1,-2.48564,1.24529,-0.39431,1.06648,1.71957,-0.60296,-2.35698,-2.31878,0.95555,-2.00420,-0.75294,-3.64680,-0.55923,0
2,-0.57025,0.65495,-0.73706,0.22444,-0.44094,-0.42596,0.44267,-1.74196,-0.06599,-0.36783,0.93065,-0.72392,0.37139,0
3,2.10165,1.46236,-0.25678,2.58908,-3.40158,2.43561,0.74701,-0.65229,-0.45497,-1.57951,2.07367,1.86435,-0.37664,1
4,3.01588,-2.50913,-2.01442,2.10296,-3.05465,5.38227,0.46846,1.57586,3.02176,-0.86271,-1.14418,-0.89712,0.61823,1


In [55]:
def generate_metadata(dataframe):

    metadata2 = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata2=metadata2.sort_values(by='tipo')
    metadata2 = metadata2.reset_index(drop=True)

    return metadata2

metadados = generate_metadata(DATA)
metadados

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,target,int64,0,0.0,2
1,v_1,float64,0,0.0,2496
2,v_2,float64,0,0.0,2495
3,v_3,float64,0,0.0,2497
4,v_4,float64,0,0.0,2494
5,v_5,float64,0,0.0,2495
6,v_6,float64,0,0.0,2497
7,v_7,float64,0,0.0,2494
8,v_8,float64,0,0.0,2494
9,v_9,float64,0,0.0,2498


In [56]:
DATA.describe()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,target
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.00000
mean,-0.011401,-0.003090,-0.521638,0.971021,-0.741765,1.086300,-0.572599,-0.758578,0.499644,-0.995828,-0.001985,-1.256229,0.012317,0.50320
std,1.677724,2.008608,1.750185,2.329727,2.561613,2.723206,1.570978,2.877406,1.605483,1.321889,1.702620,2.789631,0.990658,0.50009
min,-4.711120,-7.169570,-5.686860,-9.185750,-7.675560,-9.780180,-7.518820,-10.923700,-5.038580,-5.895320,-4.569390,-10.394380,-3.362940,0.00000
25%,-1.238582,-1.376815,-1.728730,-0.477980,-2.795507,-0.694562,-1.583137,-2.605395,-0.602595,-1.845885,-1.254970,-3.257135,-0.675635,0.00000
50%,-0.079395,0.116060,-0.644290,1.066820,-0.839305,1.292615,-0.399600,-0.812185,0.566005,-0.992885,-0.218980,-1.347365,0.011045,1.00000
75%,1.117718,1.368095,0.534890,2.559652,1.012323,2.954100,0.523932,1.180462,1.580725,-0.144530,1.122950,0.599712,0.665195,1.00000
max,5.581130,6.433950,6.043830,8.304690,8.471340,12.829340,3.487330,8.346350,6.277860,4.641370,6.901810,9.462400,3.208150,1.00000


In [51]:
# criando o melhor preditor de light GBM
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier

x_treino_final = DATA.drop('target', axis=1)
y_treino_final = DATA['target']


# Definindo o modelo
model = DecisionTreeClassifier()

# Definindo os parâmetros para o grid search
param_grid = {
    'criterion':['entropy'],
    #'splitter':['best','random'],
    #'max_depth': [2,3,4], # Profundidade máxima da árvore
    #'min_samples_split': [0.008, 0.01,0.02,0.04], # Número mínimo de amostras necessárias para dividir um nó interno.
    #'min_samples_leaf': [0.008,0.02,0.04], # Número mínimo de amostras necessárias para estar em um nó folha.
}

# Definindo o objeto GridSearchCV
grid = GridSearchCV(model,
                    param_grid,
                    cv=KFold(n_splits = 10, shuffle=False), # padrão de shuffle é False
                    verbose=2,
                    scoring='neg_log_loss',  # 'accuracy' 'precision' 'recall' 'roc_auc' 'f1'
                    n_jobs=-1,
                    return_train_score = True)

# Treinando o modelo com o grid search
grid.fit(x_treino_final, y_treino_final)

# Exibindo os melhores parâmetros encontrados pelo grid search
print("Melhores Parâmetros: ", grid.best_params_)

# Exibindo a melhor pontuação (score) atingida pelo modelo com os melhores parâmetros
print("Melhor log loss: ", grid.best_score_)




best_model = grid.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Melhores Parâmetros:  {'criterion': 'entropy'}
Melhor log loss:  -3.1638317974891725


In [52]:
estudo = pd.DataFrame(grid.cv_results_ )
estudo['mean_train_score']

0   -2.220446e-16
Name: mean_train_score, dtype: float64

In [53]:
estudo['mean_test_score']

0   -3.163832
Name: mean_test_score, dtype: float64

In [35]:
estudo.iloc[:,7:]

,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,-3.404123,-4.405335,-2.603153,-4.00485,-3.20388,-3.604365,-1.60194,-3.003638,-3.003638,-3.163832,...,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,0.0
